In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('../../data/NHGIS/nhgis0001_csv/nhgis0001_ts_geog2010_county.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9429 entries, 0 to 9428
Data columns (total 10 columns):
GISJOIN     9429 non-null object
GEOGYEAR    9429 non-null int64
DATAYEAR    9429 non-null int64
STATE       9429 non-null object
STATEA      9429 non-null int64
COUNTY      9429 non-null object
COUNTYA     9429 non-null int64
CL8AA       9429 non-null float64
CL8AAL      6286 non-null float64
CL8AAU      6286 non-null float64
dtypes: float64(3), int64(4), object(3)
memory usage: 736.7+ KB


In [5]:
df.describe()

,GEOGYEAR,DATAYEAR,STATEA,COUNTYA,CL8AA,CL8AAL,CL8AAU
count,9429.0,9429.000000,9429.000000,9429.000000,9.429000e+03,6.286000e+03,6.286000e+03
mean,2010.0,2000.000000,30.286669,103.742921,8.896779e+04,8.422344e+04,8.444773e+04
std,0.0,8.165399,15.144832,107.961774,2.904388e+05,2.781868e+05,2.786689e+05
min,2010.0,1990.000000,1.000000,1.000000,6.700000e+01,6.700000e+01,6.700000e+01
25%,2010.0,1990.000000,18.000000,35.000000,1.077800e+04,1.061150e+04,1.064850e+04
50%,2010.0,2000.000000,29.000000,79.000000,2.418100e+04,2.334200e+04,2.344050e+04
75%,2010.0,2010.000000,45.000000,133.000000,6.097318e+04,5.787375e+04,5.809650e+04
max,2010.0,2010.000000,56.000000,840.000000,9.818605e+06,9.510396e+06,9.524455e+06


In [10]:
df.head(25)

,GISJOIN,GEOGYEAR,DATAYEAR,STATE,STATEA,COUNTY,COUNTYA,CL8AA,CL8AAL,CL8AAU
0,G0100010,2010,1990,Alabama,1,Autauga County,1,34223.68,34222.0,34572.0
1,G0100030,2010,1990,Alabama,1,Baldwin County,3,98280.00,98280.0,98280.0
2,G0100050,2010,1990,Alabama,1,Barbour County,5,25417.62,25417.0,25441.0
3,G0100070,2010,1990,Alabama,1,Bibb County,7,16610.89,16470.0,16921.0
4,G0100090,2010,1990,Alabama,1,Blount County,9,39251.65,39074.0,39523.0
5,G0100110,2010,1990,Alabama,1,Bullock County,11,10977.33,10954.0,11040.0
6,G0100130,2010,1990,Alabama,1,Butler County,13,21891.61,21791.0,21970.0
7,G0100150,2010,1990,Alabama,1,Calhoun County,15,116035.05,115968.0,116183.0
8,G0100170,2010,1990,Alabama,1,Chambers County,17,36879.09,36857.0,36928.0
9,G0100190,2010,1990,Alabama,1,Cherokee County,19,19542.84,19433.0,19838.0
